In [1]:
from pycocotools.coco import COCO
import torch
from effdet import get_efficientdet_config, EfficientDet, DetBenchPredict
from effdet.efficientdet import HeadNet
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore') 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [2]:
IMAGENET_DEFAULT_MEAN = [x * 255 for x in (0.485, 0.456, 0.406)]
IMAGENET_DEFAULT_STD = [x * 255 for x in (0.229, 0.224, 0.225)]

class CustomDataset(Dataset):
    '''
      data_dir: data가 존재하는 폴더 경로
      transforms: data transform (resize, crop, Totensor, etc,,,)
    '''
    def __init__(self, annotation, data_dir, transforms):
        super().__init__()
        self.data_dir = data_dir
        # coco annotation 불러오기 (coco API)
        self.coco = COCO(annotation)
        self.predictions = {
            "images": self.coco.dataset["images"].copy(),
            "categories": self.coco.dataset["categories"].copy(),
            "annotations": None
        }
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.coco.getImgIds(imgIds=index)
        image, boxes, labels = self.load_image_and_boxes(index)

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        
        # Multi Scale
        target['img_size'] = torch.tensor([(512, 512)])
        target['img_scale'] = torch.tensor([1.])
        
        if self.transforms:
            # Transform 적용 후 Box가 없다면 있을 때 까지 반복 (default = 10)
            for i in range(10):
                sample = {
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': target['labels']
                }
                sample = self.transforms(**sample)
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  #yxyx: be warning
                    target['labels'] = torch.tensor(sample['labels'])
                    break
        
        return image, target, image_id
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())
    
    def load_image_and_boxes(self, index):
        image_id = self.coco.getImgIds(imgIds=index)
        image_info = self.coco.loadImgs(image_id)[0]
        
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image = (image - IMAGENET_DEFAULT_MEAN) / IMAGENET_DEFAULT_STD

        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        anns = self.coco.loadAnns(ann_ids)

        boxes = np.array([x['bbox'] for x in anns])
        
        labels = np.array([x['category_id'] for x in anns])
        #labels = torch.as_tensor(labels, dtype=torch.int64)

        # boxex (x_min, y_min, x_max, y_max)
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        return image, boxes, labels

def collate_fn(batch):
    return tuple(zip(*batch))

def get_test_transform():
    return A.Compose([
        A.Resize(512, 512),
        ToTensorV2(p=1.0)
    ], bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels'])
    )

In [3]:
def get_saved_model(check_dir):
    # get weight
    # url : https://github.com/rwightman/efficientdet-pytorch
    config = get_efficientdet_config('tf_efficientdet_d4')
    config.image_size = (512, 512)
    config.norm_kwargs=dict(eps=.001, momentum=.01)
    net = EfficientDet(config, pretrained_backbone=False)
    net.reset_head(num_classes=11)
    net.class_net = HeadNet(config, num_outputs=11)
    
    net = DetBenchPredict(net)
    checkpoint = torch.load(check_dir)
    net.load_state_dict(checkpoint)

    return net

In [4]:
def test_fn(data_loader, model, device):
    model.eval()
    outputs = []
    with torch.no_grad():
        for images, targets, image_ids in tqdm(data_loader):
            # gpu 계산을 위해 image.to(device)
            images = torch.stack(images)
            images = images.to(device).float()

            det = model(images)

            for i in range(0, len(det)):
                pred_scores=det[i, :, 4].cpu().unsqueeze_(0).numpy()
                condition=(pred_scores > 0.05)[0]

                pred_bboxes=det[i, :, 0:4].cpu().unsqueeze_(0).numpy()[:, condition, :]
                pred_labels=det[i, :, 5].cpu().unsqueeze_(0).numpy()[:, condition]
                pred_scores=pred_scores[:, condition]
                
                outputs.append({'boxes':pred_bboxes,
                            'scores':pred_scores,
                            'labels':pred_labels})

    return outputs

In [5]:
test_annotation = './input/data/test.json'
data_dir = './input/data'
test_dataset = CustomDataset(test_annotation, data_dir, get_test_transform())
test_data_loader=DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn, num_workers=2)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [9]:
checkpoint_path = './saved/checkpoint.pth'

model = get_model(checkpoint_path)
model.to(device)

DetBenchPredict(
  (model): EfficientDet(
    (backbone): EfficientNetFeatures(
      (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act1): SwishMe()
      (blocks): Sequential(
        (0): Sequential(
          (0): DepthwiseSeparableConv(
            (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (act1): SwishMe()
            (se): SqueezeExcite(
              (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
              (act1): SwishMe()
              (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            )
            (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affi

In [10]:
outputs = test_fn(test_data_loader, model, device)

100%|██████████| 27/27 [00:31<00:00,  1.16s/it]


In [12]:
score_threshold = 0.001

prediction_strings = []
file_names = []
coco = COCO(test_annotation)
for i, output in tqdm(enumerate(outputs)):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for box, score, label in zip(output['boxes'][0], output['scores'][0], output['labels'][0]):
        if score > score_threshold:
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0]) + ' ' + str(box[1]) + ' '\
                + str(box[2]) + ' ' + str(box[3]) + ' '
                
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])

submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(f'submission_alldata_68_80s.csv', index=None)
print(submission.head())

322it [00:00, 3209.21it/s]

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


837it [00:00, 3587.16it/s]

                                    PredictionString              image_id
0  8 0.9849696 171.14967 233.77429 464.5365 472.3...  batch_01_vt/0021.jpg
1  8 0.9930957 275.79028 204.74718 484.53833 431....  batch_01_vt/0028.jpg
2  1 0.81985927 52.3862 160.48149 373.3861 381.15...  batch_01_vt/0031.jpg
3  8 0.9130863 35.143234 141.24197 388.95007 368....  batch_01_vt/0032.jpg
4  8 0.9920412 0.5099106 254.92639 222.95645 479....  batch_01_vt/0070.jpg
